In [ ]:
# Import des librairies 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from time import sleep
import csv

In [ ]:
# Connexion a LinkedIn

# Ouverture de Chrome et connexion a LinkedIn
driver = webdriver.Chrome()
sleep(2)
url = 'https://www.linkedin.com/login'
driver.get(url)
print('Initialisation driver')
sleep(2)

# Import username et password. Il faut un fichier local avec les infos de connexions
credential = open('credentials.txt')
line = credential.readlines()
username = line[0]
password = line[1]
print('Succès import credentials')
sleep(2)

# Task 1.2: Key in login credentials
email_field = driver.find_element(By.NAME, "session_key")
email_field.send_keys(username)
print('Succès insertion email')
sleep(3)

password_field = driver.find_element(By.NAME, "session_password")
password_field.send_keys(password)
print('Succès insertion password')
sleep(2)

# Soumission formulaire
# Plus besoin de cliquer on dirait ? Je garde ca en commentaire si besoin
#signin_field = driver.find_element(By.XPATH, '//*[@id="organic-div"]/form/div[3]/button')
#signin_field.click()

print('Connexion a LinkedIn avec succès')

In [ ]:
# Recherche d'un profil. Ici Data Engineer

# Accès a la barre de recherche
search_field = driver.find_element(By.XPATH, '//*[@id="global-nav-typeahead"]/input')

# Insertion du profil recherché

#search_query = 'Data engineer'
search_query = input('Quel profil scraper? ')
search_field.send_keys(search_query)

# Lancement de la recherche
search_field.send_keys(Keys.RETURN)

print('Recherche des profils avec succès')

In [ ]:
link = driver.find_element(By.LINK_TEXT, 'Voir tous les résultats d’offres d’emploi pour : France')
link.click()

In [ ]:
# Task 3: Scrape the URLs of the profiles

#Pour des besoin pratique, je donne l'URL qui m'interesse ici (Pour ma recherche personnel, je veux un poste de Data Engineer en full remote):
#Il est plus facile de donner l'URL avec les paramètres que de passer par le script pour accéder aux menus déroulant.

url = 'https://www.linkedin.com/jobs/search/?currentJobId=3140318840&f_TPR=r604800&f_WT=2&keywords=data%20engineer&sortBy=R'
driver.get(url)

# Fonction d'extraction des url des différents profils proposer pour accéder ensuite au détail des annonces via ces URL
def GetURL():
    page_source = BeautifulSoup(driver.page_source)
    profiles = page_source.find_all('a', class_ = 'disabled ember-view job-card-container__link job-card-list__title')
    all_profile_URL = []
    for profile in profiles:
        profile_ID = profile.get('href')
        profile_URL = "https://www.linkedin.com" + profile_ID
        if profile_URL not in all_profile_URL:
            all_profile_URL.append(profile_URL)
    return all_profile_URL

# Ouvrir chaque URL et récupérer les information de chaque profil

input_page = int(input('Combien de page scraper?: '))
URLs_all_page = []
for page in range(input_page):
    URLs_one_page = GetURL()
    sleep(2)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll a la fin de la page
    sleep(3)
    next_button = driver.find_element(By.CLASS_NAME, "artdeco-button__text")
    driver.execute_script("arguments[0].click();", next_button)
    URLs_all_page = URLs_all_page + URLs_one_page
    sleep(2)

print('Scraping des url avec succès')

In [ ]:
#Ici, on va accéder a chaque URL qu'on a concaténer dans URLs_all_page, et récupérer le nom de l'entreprise, le profil, 
#la géolocalication et le type de poste pour s'assurer qu'on est bien sur du full remote
with open('output.csv', 'w',  newline = '') as file_output:
    headers = ['Company Name', 'Job Title', 'Location', 'URL']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for linkedin_URL in URLs_all_page:
        driver.get(linkedin_URL)
        print('- Accès au profil via URL: ', linkedin_URL)

        page_source = BeautifulSoup(driver.page_source, "html.parser")

        info_div = page_source.find('div',{'class':'p5'})
        span_name = info_div.find('span', {'class':'jobs-unified-top-card__company-name'})
        company_name = span_name.find('a').get_text().strip()
        print('--- Company name: ', company_name)
        location = info_div.find('span', {'class':'jobs-unified-top-card__bullet'}).get_text().strip() #Remove unnecessary characters 
        print('--- Profile location is: ', location)
        workplace_type = info_div.find('span', {'class', 'jobs-unified-top-card__workplace-type'}).get_text().strip()
        print('--- Workplace type is :', workplace_type)
        title = info_div.find('h1').get_text().strip()
        print('--- Profile title is: ', title)
        writer.writerow({headers[0]:company_name, headers[1]:location, headers[2]:title, headers[3]:linkedin_URL})
        print('\n')

print('Extraction des données OK!')